In [1]:
import pandas as pd
import pickle
from BayDS.lib.training import *
from sklearn.model_selection import GroupKFold, KFold

In [2]:
data = pd.read_pickle("temp/data.pkl")

In [3]:
data

,userid,field0,field1,field2,field3,field4,field5,field6,field7,field8,...,field27_count_groupby_field29,userid_count_groupby_field29,field5_count_groupby_userid,field7_count_groupby_userid,field8_count_groupby_userid,field10_count_groupby_userid,field12_count_groupby_userid,field26_count_groupby_userid,field27_count_groupby_userid,field29_count_groupby_userid
orderid,,,,,,,,,,,,,,,,,,,,,
1000000,10d654494cbe97bbb25d51ead2600679aff9e097924add...,0,-0.626508119430350,11,12,1,1,0,1,1,...,154699,1,1,1,1,1,1,1,1,1
1000001,4aafc0391f72bbcf60537aece62923baf9ce644b64ac36...,144,-0.393794036775907,5,7,2,0,0,2,1,...,83311,1,1,2,2,2,1,1,1,1
1000002,bac8ffef46348f587c8d17137ab01fb24aef21547c647d...,134,-0.548936758545536,2,3,2,0,0,1,1,...,2162,1,1,2,2,1,1,2,2,1
1000003,0392247b4b87674aba2c32bf2292b105771a6a376871be...,0,-0.238651315006278,10,11,1,1,3,2,1,...,154699,1,1,1,3,2,1,2,2,1
1000004,d1aeefef311bbeb4bd84876c8d49421f276674527d5578...,0,-0.704079480315165,8,11,1,1,0,1,1,...,199779,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455006,e54d8c57b0d2b64d19c59d5068372758067921f06101e2...,0,2.553917676847042,8,8,1,1,10,2,0,...,199779,1,1,1,1,1,1,1,1,1
455007,5af2c8b53a965915f496971f0f105393f7f9c9b0931e6b...,0,-0.238651315006278,9,9,1,1,1,1,1,...,199779,1,1,1,1,1,1,1,1,1
455008,255bfc6ac2f9c98d7f86e59bd72a682e7a50c2db610749...,0,-0.161079954121464,10,11,1,1,0,1,1,...,154699,1,1,1,1,1,1,1,1,1


In [4]:
exclude_features_from_training = ['indicator_goal21', 'indicator_goal22', 'indicator_goal23',
       'indicator_goal24', 'indicator_goal25', 'goal21', 'goal22', 'goal23',
       'goal24', 'goal25', 'goal1','userid',]

In [5]:
X = data[data['goal1']>=0].drop(['goal1'], axis=1)
y = data[data['goal1']>=0]['goal1']
test=data[data['goal1']<0]
train_features = [f for f in data.columns if f not in exclude_features_from_training]

In [10]:
results = {}

In [ ]:
import datetime

NFOLDS = 5
folds = GroupKFold(n_splits=NFOLDS)

for max_depth in range(2,20,2):
    print(f"**********\n Max_depth={max_depth}\n")
    params = {
              'objective': 'binary',
              "metric": 'auc',
              "verbosity": -1,
              'random_state':0,
              'max_depth': max_depth
             }
    train_options = {
            "model_type":'lgb',
            "params": params,
            "eval_metric":'auc',
            'early_stopping_rounds': 100,
            'n_estimators': 500,
            'averaging': 'rank',
            'use_groups': False,
            'fold_name': folds.__class__.__name__,
            'n_splits': NFOLDS,

        }


    result_dict = train_model_classification_vb( X=X, 
                                                 X_test=test, 
                                                 columns=train_features,
                                                 y=y, 
                                                 params=params, folds=folds,
                                                 model_type=train_options['model_type'], 
                                                 plot_feature_importance=True,
                                                 verbose=100, early_stopping_rounds=train_options['early_stopping_rounds'],
                                                 n_estimators=train_options['n_estimators'], 
                                                 averaging=train_options['averaging'],
                                                 groups=X['userid'],
                                                 n_jobs=-1)
    results[max_depth] = result_dict
    pickle.dump(results, open('temp/max_depth.pkl','wb'))
    
    t = datetime.datetime.now().strftime("%m-%d-%H-%M")
    test['proba'] = result_dict['prediction']
    sub = pd.read_csv('data/onetwotrip_challenge_sub1.csv')
    sub['proba'] = test['proba'] / test['proba'].max()
    sub.reset_index(drop=True).set_index('orderid').to_csv(f'temp/submission_depth={max_depth}-{t}.csv')

**********
 Max_depth=2

Fold 1 started at Tue Dec 17 17:27:09 2019
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.732959	training's auc: 0.732959	valid_1's auc: 0.676344	valid_1's auc: 0.676344
[200]	training's auc: 0.75345	training's auc: 0.75345	valid_1's auc: 0.673913	valid_1's auc: 0.673913
Early stopping, best iteration is:
[114]	training's auc: 0.736893	training's auc: 0.736893	valid_1's auc: 0.67697	valid_1's auc: 0.67697
Fold 2 started at Tue Dec 17 17:28:03 2019
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.721551	training's auc: 0.721551	valid_1's auc: 0.718717	valid_1's auc: 0.718718
[200]	training's auc: 0.743946	training's auc: 0.743946	valid_1's auc: 0.719524	valid_1's auc: 0.719524
[300]	training's auc: 0.75951	training's auc: 0.75951	valid_1's auc: 0.719195	valid_1's auc: 0.719195
Early stopping, best iteration is:
[227]	training's auc: 0.748058	training's auc: 0.748058	valid_1's auc: 0.7214

CV mean score: 0.6923, std: 0.0172.
**********
 Max_depth=10

Fold 1 started at Tue Dec 17 17:46:02 2019
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.89605	training's auc: 0.89605	valid_1's auc: 0.669423	valid_1's auc: 0.669423
Early stopping, best iteration is:
[41]	training's auc: 0.831316	training's auc: 0.831316	valid_1's auc: 0.675621	valid_1's auc: 0.675621
Fold 2 started at Tue Dec 17 17:46:54 2019
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.895597	training's auc: 0.895597	valid_1's auc: 0.712549	valid_1's auc: 0.71255
Early stopping, best iteration is:
[39]	training's auc: 0.822271	training's auc: 0.822271	valid_1's auc: 0.720132	valid_1's auc: 0.720131
Fold 3 started at Tue Dec 17 17:47:46 2019
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.897868	training's auc: 0.897868	valid_1's auc: 0.692066	valid_1's auc: 0.692066
Early stopping, best iteration is:
[29

In [12]:
CV mean score: 0.6925, std: 0.0178.
sub.reset_index(drop=True).set_index('orderid').to_csv(f'temp/submission_depth={max_depth}-{t}.csv')

In [21]:
for k,v in results.items():
    print (k, v['scores'])

0 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
1 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
2 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
3 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
4 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
5 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
6 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
7 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
8 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.6805495522257086, 0.6833038734198107]
9 [0.6764080531918433, 0.7253302297489943, 0.7011452480307342, 0.68054955